In [1]:
import gzip
import json
import pandas as pd
import numpy as np
from os.path import join, isfile
from os import listdir
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import string

In [2]:
def get_json(fname, fpath='../../wos_paper/wos_db_benchmark/benchmarking/'):
    fname = join(fpath, fname)
    if fname[-2:] == 'gz':
        with gzip.open(fname, "rb") as f:
            data = json.loads(f.read(), encoding="utf-8")
    else:
        with open(fname, 'r') as fp:
            data = json.load(fp)
    return data

def get_json_fnames(sourcepath='../../wos_paper/wos_db_benchmark/benchmarking/'):
    only_json = [f for f in listdir(sourcepath) if 
                     isfile(join(sourcepath, f)) 
                     and f[-4:] == 'json']
    return only_json


def get_result_fnames(sourcepath='../results/arango/'):
    r = [f for f in listdir(sourcepath) if 
                     isfile(join(sourcepath, f)) 
                     and f[-7:] == 'json.gz' and 'result' in f and 'limit' not in f]
    return r

def get_content(sql_list, fpath, how='sql'):
    report = []
    if how == 'sql':
        limit_tokens = 3
    elif how == 'arango':
        limit_tokens = 3
    else:
        limit_tokens = 3
    for f in sql_list:
        coded = f.split('_')
        limit = -1 if len(coded) < limit_tokens else int(coded[-1].split('.')[0])
        content = get_json(f, fpath)
        report += [(coded[0], limit, f, content)]
    return report

def get_report_gzipped(flist, fpath):
    acc = []
    for fname in flist:
        qname = fname.split('/')[0]
        data = get_json(fname, fpath)
        acc += [(qname, data)] 
    return acc

def get_mysql_times(sql_report):
    times_stat = []
    for item in sql_report:
        key, limit, f, content = item
        if 'elapsed' in content:
            times_stat += [(*item[:-1], content['elapsed'])]
        else:
            times_stat += [(*item[:-1], content['elapsed_1978'])]
    return times_stat

def get_arango_times(reports):
    times = [(*item[:-1], [sum(x.values()) for x in item[-1]]) for item in reports]
    times_stat = [(*item[:-1], np.mean(item[-1]), np.std(item[-1])) for item in times]
    return times_stat

In [3]:
sql_list = sorted(get_json_fnames())
sql_report = get_content(sql_list, '../../wos_paper/wos_db_benchmark/benchmarking/')

In [4]:
flist = sorted(get_result_fnames())
arango_data = get_report_gzipped(flist, '../results/arango/')

# Query 1

In [5]:
q1_sql = pd.DataFrame(sql_report[1][-1]['results_1978'], columns=['year', 'title', 'count'])
q1_arango = pd.DataFrame([{**item['journal'], **{'count': item['number_pubs']}} for item in arango_data[0][1]])

# q1_arango groupby by title for comparison
q1_arango_condensed = q1_arango.groupby('title')['count'].sum().reset_index().rename(columns={0:'count'})
q1_arango_condensed.sort_values('count', ascending=False).head()

,title,count
2836,FEDERATION PROCEEDINGS,7287
1528,BULLETIN OF THE AMERICAN PHYSICAL SOCIETY,5895
1973,CLINICAL RESEARCH,5208
11,ABSTRACTS OF PAPERS OF THE AMERICAN CHEMICAL S...,4514
1435,BRITISH MEDICAL JOURNAL,3597


In [6]:
q1_cmp = pd.merge(q1_sql, q1_arango, on='title', how='outer', suffixes=['_sql', '_arango'])
q1_cmp2 = pd.merge(q1_sql, q1_arango_condensed, on='title', how='outer', suffixes=['_sql', '_arango'])
q1_cmp2 = q1_cmp2.sort_values('count_arango', ascending=False)
q1_cmp2['cmp'] = (q1_cmp2['count_sql'] == q1_cmp2['count_arango'])
q1_cmp2[['title', 'count_sql', 'count_arango', 'cmp']].head(12)

,title,count_sql,count_arango,cmp
0,FEDERATION PROCEEDINGS,7287.0,7287,True
1,BULLETIN OF THE AMERICAN PHYSICAL SOCIETY,5895.0,5895,True
2,CLINICAL RESEARCH,5208.0,5208,True
3,ABSTRACTS OF PAPERS OF THE AMERICAN CHEMICAL S...,4514.0,4514,True
4,BRITISH MEDICAL JOURNAL,3597.0,3597,True
5,LANCET,3189.0,3189,True
6,TLS-THE TIMES LITERARY SUPPLEMENT,3062.0,3062,True
7,NATURE,2990.0,2990,True
8,TRANSACTIONS-AMERICAN GEOPHYSICAL UNION,2812.0,2812,True
9,DOKLADY AKADEMII NAUK SSSR,2339.0,2339,True


# Query 2

In [7]:
puncts = list(string.punctuation)
stop_words_nltk = set(stopwords.words('english'))
numerics = set([str(i) for i in range(100)])
all_stops = list(set(puncts) | stop_words_nltk | numerics)
# remove stopword from sql results
q2_sql = pd.DataFrame(sql_report[2][-1]['frequent_words'], columns=['uword', 'count'])
q2_sql_mod = q2_sql.loc[~q2_sql.uword.isin(all_stops)]

#remove '.2.' from arango results
q2_arango = pd.DataFrame(arango_data[1][1])
q2_arango = q2_arango.loc[~q2_arango.uword.apply(lambda x: x.startswith('.'))]


q2_cmp = pd.merge(q2_sql_mod, q2_arango, on='uword', how='outer', 
                  suffixes=['_sql', '_arango'])
# define error
q2_cmp['err'] = (q2_cmp['count_sql'] - q2_cmp['count_arango'])/q2_cmp['count_arango']

In [8]:
# define ranks
q2_cmp = q2_cmp.sort_values('count_sql', 
                            ascending=False).\
        reset_index().rename(columns={'index': '_'}).\
        reset_index().rename(columns={'index': 'rank_sql'})
q2_cmp = q2_cmp.sort_values('count_arango', 
                            ascending=False).\
        reset_index().rename(columns={'index': '_'}).\
        reset_index().rename(columns={'index': 'rank_arango'})
q2_cmp = q2_cmp.drop('_', axis=1)
q2_cmp.sort_values('count_sql', ascending=False).head(20)

,rank_arango,rank_sql,uword,count_sql,count_arango,err
0,0,0,effect,242666.0,242312.0,0.001461
1,1,1,study,189789.0,189016.0,0.004090
2,2,2,effects,185911.0,185526.0,0.002075
3,3,3,studies,185596.0,184634.0,0.005210
4,4,4,new,148332.0,147154.0,0.008005
5,5,5,analysis,114563.0,113725.0,0.007369
6,6,6,synthesis,107862.0,105913.0,0.018402
7,7,7,human,101709.0,101340.0,0.003641
10,10,8,structure,99601.0,97406.0,0.022535
8,8,9,properties,99019.0,98039.0,0.009996


# Query 3

In [9]:
q3_sql = pd.DataFrame(sql_report[3][3]['results'][:], 
                      columns=['last_name', 'first_name', 'cnt']).sort_values(['cnt', 'last_name', 'first_name'], 
                                                                              ascending=False)
q3_sql = q3_sql[['last_name', 'first_name', 'cnt']]

q3_arango = pd.DataFrame(arango_data[2][1]).sort_values(['cnt', 'last_name', 'first_name'], ascending=False)
q3_arango = q3_arango.loc[(q3_arango.last_name != '') & (q3_arango.first_name != '')]
q3_arango = q3_arango[['last_name', 'first_name', 'cnt']]

In [10]:
q3_cmp = pd.merge(q3_sql, q3_arango, on=['last_name', 'first_name'], suffixes=['_sql', '_arango'])

In [11]:
q3_cmp.head(20)

,last_name,first_name,cnt_sql,cnt_arango
0,SCHLEYER,PV,16,16
1,SACTON,J,14,14
2,MARICIC,S,12,12
3,WILSON,R,11,11
4,ROBSON,EB,11,11
5,PERNIS,B,11,11
6,MOLINA,R,11,11
7,MEBOLD,U,11,11
8,LUMBROSO,H,11,11
9,DAVIS,DH,11,11


# Query 4

In [12]:
q4_sql = pd.DataFrame(sql_report[4][-1]['results']).rename(columns={'A': 'wosid'})
q4_arango = pd.DataFrame([{'ratio': dd['f'], 'wosid': w} for dd in arango_data[3][-1] for w in dd['ids']])
q4_cmp = pd.merge(q4_sql[['wosid', 'ratio']], q4_arango, on='wosid', how='left', suffixes=['_sql', '_arango'])
q4_cmp['err'] = abs((q4_cmp['ratio_sql'] - q4_cmp['ratio_arango'])/q4_cmp['ratio_arango'])
q4_cmp['flag'] = (q4_cmp['err'] > 1e-3)
q4_cmp.head(20)

,wosid,ratio_sql,ratio_arango,err,flag
0,WOS:A1978FV54600117,126.0000,126.000000,0.000000e+00,False
1,WOS:A1978FW60600008,103.0000,103.000000,0.000000e+00,False
2,WOS:A1978EV31700040,85.0000,85.000000,0.000000e+00,False
3,WOS:A1978EP90300642,75.0000,75.000000,0.000000e+00,False
4,WOS:A1978EK02600050,71.0000,71.000000,0.000000e+00,False
5,WOS:A1978EU89401637,70.0000,70.000000,0.000000e+00,False
6,WOS:A1978EW70500022,67.8000,67.800000,0.000000e+00,False
7,WOS:A1978EG81500237,67.0000,67.000000,0.000000e+00,False
8,WOS:A1978EP04700423,63.0000,63.000000,0.000000e+00,False
9,WOS:A1978EH33500016,60.0000,60.000000,0.000000e+00,False


In [13]:
sum(q4_cmp.flag)

7

In [14]:
q4_cmp.loc[q4_cmp.flag]

,wosid,ratio_sql,ratio_arango,err,flag
1286,WOS:A1978EL06300012,11.1000,11.000000,0.009091,True
2283,WOS:A1978EQ68300006,8.8000,8.600000,0.023256,True
3365,WOS:A1978EP75000100,7.0000,6.666667,0.050000,True
3818,WOS:A1978EM76300021,6.9091,6.818182,0.013335,True
4670,WOS:A1978EM59000031,6.0000,5.666667,0.058824,True
5597,WOS:A1978GU98300040,5.6667,5.333333,0.062506,True
5758,WOS:A1978EY97900005,5.5000,5.416667,0.015385,True


# Query 5 : only arango ready

In [15]:
ww = arango_data[4][1]

In [16]:
jas = sorted([x['ja'] for x in ww])
jbs = set([y['jb'] for x in ww for y in x['stats']])
jall = sorted(set(jas) | jbs)
print(len(jas), len(jbs), len(jall))

5506 5148 5859


In [17]:
acc = []
jas = [item['ja'] for item in ww]
for item in ww[:]:
    arr = [x['s'] for x in item['stats']]
    ixs = [x['jb'] for x in item['stats']]
    acc += [pd.Series(arr, index=ixs)]
q5_arango = pd.concat(acc, axis=1, sort=True)
q5_arango = q5_arango.rename(columns=dict(zip(range(len(jas)), jas)))

In [18]:
q5_arango.head()

,0001-0782,0001-1096,0001-1096,0001-1198,0001-1452,0001-1479,0001-1541,0001-2351,0001-2491,0001-2491,...,1756-1833,1756-1833,1862-6300,1882-3351,4791-0142,8750-2836,8750-3735,8750-7587,8755-1209,8756-4211
,5.0,NaN,30.0,NaN,3.0,NaN,8.0,4.0,NaN,NaN,...,2.0,NaN,1.0,NaN,NaN,NaN,1.0,24.0,189.0,NaN
0001-0782,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0001-1096,NaN,NaN,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
0001-1169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0001-1198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
q5_arango.shape

(5148, 5506)

In [20]:
sql_report[5][-1]['results'][:3]

[['WOS:A1978EQ77900041',
  'WOS:A1977FK40800060',
  'WOS:A1979GU56800033',
  'WOS:A1979GU56800020',
  'WOS:A1980KG46700021',
  'WOS:A1980KT32200049',
  923809],
 ['WOS:A1978EU94600059',
  'WOS:A1978FW57000020',
  'WOS:A1979HD90700009',
  'WOS:A1979HX65300041',
  'WOS:A1980KQ38200012',
  'WOS:A1980KT80300054',
  518806],
 ['WOS:A1978EU94600057',
  'WOS:A1978FU73000036',
  'WOS:A1979HA49100046',
  'WOS:A1979JA28200014',
  'WOS:A1980KA82000001',
  'WOS:A1980KL70800037',
  465969]]

In [21]:
#q6
arango_data[-1]

('query6_result_batch_0.json.gz',
 [{'pset5': 65894, 'ids': ['WOS:A1978EK02600051']},
  {'pset5': 65776, 'ids': ['WOS:A1978FM59000001']},
  {'pset5': 57242, 'ids': ['WOS:A1978FZ94800026']},
  {'pset5': 50047, 'ids': ['WOS:A1978GJ03600004']},
  {'pset5': 48144, 'ids': ['WOS:A1978FJ22100001']},
  {'pset5': 47855, 'ids': ['WOS:A1978FZ79400013']},
  {'pset5': 47327, 'ids': ['WOS:A1978FD98400005']},
  {'pset5': 47093, 'ids': ['WOS:A1978FX80100019']},
  {'pset5': 46632, 'ids': ['WOS:A1978EK02600050']},
  {'pset5': 45644, 'ids': ['WOS:A1978FB74800005']},
  {'pset5': 45206, 'ids': ['WOS:A1978EJ61900001']},
  {'pset5': 43704, 'ids': ['WOS:A1978GG67000001']},
  {'pset5': 39219, 'ids': ['WOS:A1978EW70500023']},
  {'pset5': 34698, 'ids': ['WOS:A1978EW70500022']},
  {'pset5': 32765, 'ids': ['WOS:A1978FW60600008']},
  {'pset5': 30799, 'ids': ['WOS:A1978FL69000004']},
  {'pset5': 30487, 'ids': ['WOS:A1978GC25200056']},
  {'pset5': 23979, 'ids': ['WOS:A1978GA84500005']},
  {'pset5': 22215, 'ids': ['WO